### This notebook uses Natural Language Processing and Machine Learning to predict gender of scientists

### Import packages

In [ ]:
import random 
from nltk.corpus import names 
import nltk 
import pandas as pd 
import numpy as np

In [ ]:
# Read the excel file into a dataframe
file = pd.read_excel('C:/Users/nikhi/Desktop/Summer 2020/RA Work/RA NBER/listofauthorswithfirstname_Encoded/listofauthorswithfirstname_Encoded.xlsx')

### Define functions that use natural language processing and machine learning

In [ ]:
def gender_features(word): 
    # tokenizes parts of the scientist name
    return {'last_2letters':word[-2:], 'last_letter':word[-1], 'last_3letter':word[-3:],'last_4letter':word[-4:],'first_letter':word[0], 'first_2letter':word[:2]} 
  
def get_prediction(country, male, female):
    # preparing a list of names and corresponding gender labels
    labeled_names = ([(name.split(' ')[0], 'male') for name in male['given_name']]+
             [(name.split(' ')[0], 'female') for name in female['given_name']])

    n = len(male) + len(female) 
  
    random.shuffle(labeled_names) 
  
    # we use the feature extractor to process the names data. 
    featuresets = [(gender_features(n), gender)  
               for (n, gender)in labeled_names] 
  
    # Divide the resulting list of feature 
    # sets into a training set and a test set. 
    train_set, test_set = featuresets[:n-100], featuresets[n-100:] 
  
    # The training set is used to train a new "naive Bayes" classifier. 
    classifier = nltk.NaiveBayesClassifier.train(train_set)   
    print(country, nltk.classify.accuracy(classifier, train_set))
    print(country, nltk.classify.accuracy(classifier, test_set))
    #classifier.show_most_informative_features(200)

    return classifier

### Use above functions to predict of scientists

In [ ]:
# list of all unique countries in the dataset
countries = file.affiliation_country.unique()
pred = [] # gender prediction
acc = [] # accuracy of prediction
for country in countries:
    # list of all male names in that country
    male = file[(file['GENDER']==1) & (file['Country']==country)]

    # list of all female names in that country
    female = file[(file['GENDER']==2) & (file['Country']==country)]

    # list of names with unknown gender
    unk = file[(file['GENDER']==0) & (file['Country']==country)]

    # get the gender predicter object for that country
    c = get_prediction(country, male, female)
    for name in unk['given_name']:
        # find the gender of unknown names
        pred.append([country, name, c.classify(gender_features(str(name).split(' ')[0]))])

In [ ]:
pred